# Skripsi v1

## Integrasi SMOTE dan Neural Network

100% jupyter Notebook Cell

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [126]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from bayes_opt import BayesianOptimization as bayesopt

import tool

In [128]:
dataset_kc1 = pd.read_csv("Dataset/clean/KC1.csv")
dataset_kc2 = pd.read_csv("Dataset/clean/KC2.csv")
dataset_pc1 = pd.read_csv("Dataset/clean/PC1.csv")
dataset_pc3 = pd.read_csv("Dataset/clean/PC3.csv")
dataset_pc4 = pd.read_csv("Dataset/clean/PC4.csv")

dataset = [data_kc1, data_kc2, data_pc1, data_pc3, data_pc4]
dataset_string = ["KC1", "KC2", "PC1", "PC3", "PC4"]

Setiap dataset memiliki column class berbeda-beda. dibawah ini proses mengubah kolom kelas menjadi seragam ("Defect")

In [129]:
i = -1
for data in dataset:
    i += 1
    dataset[i] = data.rename(columns = {data.columns[-1]:'defect'})
    print(dataset_string[i])
    display(dataset[i].head(3))

KC1


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defect
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,True
2,83.0,11.0,1.0,11.0,171.0,927.89,0.04,23.04,40.27,21378.61,...,65,10,6,0,18.0,25.0,107.0,64.0,21.0,True


KC2


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defect
0,1.1,1.4,1.4,1.4,1.3,1.3,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,3.0,1.0,1.0,1.0,1.0,0.0,0.00,0.00,0.00,0.00,...,1,0,0,0,1.0,0.0,1.0,0.0,1.0,False
2,14.0,2.0,1.0,2.0,22.0,88.0,0.17,5.79,15.21,509.14,...,8,0,1,0,9.0,7.0,13.0,9.0,3.0,False


PC1


,loc,v(g),ev(g),iv(G),N,V,L,D,I,E,...,lOCode,lOComment,locCodeAndComment,lOBlank,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defect
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,True
2,91.0,9.0,3.0,2.0,318.0,2089.21,0.04,27.68,75.47,57833.24,...,80,44,11,31,29.0,66.0,192.0,126.0,17.0,True


PC3


,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,defect
0,16,13,1,6,11,20,7,0.24,8,2.5,...,17,0.12,68,81,31,21,57,42.50,29,False
1,1,13,5,0,0,22,7,0.37,10,2.2,...,23,0.33,38,53,19,19,21,0.00,19,False
2,8,3,1,0,1,4,2,0.14,2,2.0,...,6,0.08,32,38,19,13,24,6.67,14,False


PC4


,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,defect
0,17,11,5,2,8,20,6,0.25,10,2,...,25,0.11,53,49,23,12,57,31.25,24,False
1,2,9,3,0,1,16,5,0.56,6,2,...,14,0.36,13,24,7,14,14,10.00,9,False
2,2,5,1,1,1,6,3,0.17,2,3,...,7,0.13,16,28,9,14,23,10.53,18,False


proses min-max Scaller untuk mengubah variant distribusi

In [130]:
scaller = MinMaxScaler()
i = -1
for data in dataset:
    i += 1
    dataset[i].loc[:, dataset[i].columns != dataset[i].columns[-1]] = scaller.fit_transform(data.loc[:, data.columns != data.columns[-1]])
    print(dataset_string[i])
    display(dataset[i].loc[:, dataset[i].columns[0]:dataset[i].columns[4]].head(3))

KC1


,loc,v(g),ev(g),iv(g),n
0,0.000348,0.009091,0.016,0.009091,0.001175
1,0.000000,0.000000,0.000,0.000000,0.000904
2,0.285714,0.227273,0.000,0.227273,0.154611


KC2


,loc,v(g),ev(g),iv(g),n
0,0.000078,0.002235,0.003226,0.002817,0.000075
1,0.001570,0.000000,0.000000,0.000000,0.000000
2,0.010204,0.005587,0.000000,0.007042,0.005275


PC1


,loc,v(g),ev(g),iv(G),N
0,0.001827,0.002963,0.003279,0.003279,0.000108
1,0.001661,0.000000,0.000000,0.000000,0.000000
2,0.151163,0.059259,0.016393,0.008197,0.113865


PC3


,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS
0,0.207792,0.020408,0.041667,0.142857,0.141026
1,0.012987,0.020408,0.208333,0.000000,0.000000
2,0.103896,0.003401,0.041667,0.000000,0.012821


PC4


,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS
0,0.188889,0.053763,0.208333,0.029412,0.102564
1,0.022222,0.043011,0.125000,0.000000,0.012821
2,0.022222,0.021505,0.041667,0.014706,0.012821


Pemodelan